TODO: 

2. Check taxonomy class, the constructor __init__ get inputs that are unused. 
3. Verify that we can load and create another notebook for Viz. 
4. Create another classifier for "hybrid" taxonomy
5. Cleanup commented functios in TissueGraph.py (they are mostly related to hybrid taxonomy) 
6. Cleanup dependencies and unused methods from Utils
7. Viz needs to support Geoms as list
8. update Viz code (mostly comments)
9. Finish configuring Sphinx
10. Push to github




# Create and construct a TMG object from multiple slices

This example notebooks performs basic TMG analysis operations. The "input" are the per slice files generated by Processing module. 

In this example, the Taxonomy/Classifiers are not given and are created on the fly based on optimal leiden. 

Overall, the steps in the analysis are: 
1. Load and create an empty TMG object
2. Create a cell layer. This loads, normalizes, creates graphs, etc, but does not classify
3. Unsupervized learning using optimal leiden to create classifier and taxonomy
4. Create iso-zone layer
5. Classify cellular neighborhoods into regions using topic modeling
6. save to file

### Import and create an empty TMG

In [11]:
import igraph
import matplotlib.pyplot as plt 

from dredFISH.Analysis.TissueGraph import *
from dredFISH.Visualization.Viz import *
from dredFISH.Analysis.Classification import *

import importlib
from dredFISH.Analysis import TissueGraph
importlib.reload(TissueGraph)
from dredFISH.Analysis.TissueGraph import *

from dredFISH.Analysis import Classification
importlib.reload(Classification)
from dredFISH.Analysis.Classification import *


from dredFISH.Utils import tmgu
importlib.reload(tmgu)

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')

In [12]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

total 81M
-rw-r--r-- 1 zach     wollmanlab  16M Jun  9 14:55 DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_matrix.csv
-rw-r--r-- 1 zach     wollmanlab  20M Jun  9 14:55 DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_metadata.csv
drwxrwxrwx 4 zach     wollmanlab   48 Jun 10 08:52 ..
drwxrwxr-x 2 rwollman wollmanlab 4.0K Jul  6 15:01 .
-rw-rw-r-- 1 rwollman wollmanlab  167 Jul  7  2022 TMG.json
-rw-rw-r-- 1 rwollman wollmanlab  40M Jul  7  2022 cells.h5ad
-rw-rw-r-- 1 rwollman wollmanlab 6.8M Jul  7  2022 isozones.h5ad
-rw-rw-r-- 1 rwollman wollmanlab 646K Jul  7  2022 region.h5ad
-rw-r--r-- 1 fangming wollmanlab    3 Jul  7  2022 Taxonomy_clusters.csv
-rw-r--r-- 1 fangming wollmanlab    3 Jul  7  2022 Taxonomy_topics.csv
{"layers_graph": [[0, 1], [0, 2]], "layer_taxonomy_mapping": {"0": 0, "1": 0, "2": 1}, "Taxonomies": ["clusters", "topics"], "Layers": ["cells", "isozones", "region"]}

In [4]:
%%time
TMG = TissueMultiGraph(basepath=basepth, 
                       redo=False, # load from existing ones
                      ) 

/bigstore/GeneralStorage/Data/dredFISH/Dataset1/cells.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1/isozones.h5ad
/bigstore/GeneralStorage/Data/dredFISH/Dataset1/region.h5ad


KeyError: 3

In [5]:
# adata = anndata.read('/bigstore/GeneralStorage/Data/dredFISH/Dataset1/cells.h5ad')
# adata
# "SG" in adata.obsp.keys()

### Create a cell layer
Creating a cell layer, load data from file, normalizes and creates an unclassified tissue graph

In [7]:
%%time
TMG.create_cell_layer(metric = 'cosine')
logging.info(f"TMG has {len(TMG.Layers)} Layers")

NameError: name 'TMG' is not defined

### Create Geometries

In [8]:
%%time
# TMG.add_geoms()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


### Create OptLeiden classifier and train it using cell layer

In [9]:
%%time
# Create the classifier
optleiden = OptimalLeidenKNNClassifier(TMG.Layers[0])
# train the classifier
optleiden.train(opt_res=11.5, opt_params={'iters':10, 'n_consensus':1})
# use the classifier to create types and add them to TMG using the Taxonomy created on the fly by the classifier
type_vec = optleiden.classify()
TMG.add_type_informations(0, type_vec, optleiden.tax)

NameError: name 'TMG' is not defined

### Create isozone layer

In [10]:
TMG.create_isozone_layer()
logging.info(f"TMG has {len(TMG.Layers)} Layers")

NameError: name 'TMG' is not defined

### Create regions layer
To create regions, we first create a new classifier that works on local cell environments (local type abundace). Classify cells based on their regions and use this type to create a new layer. 

In [ ]:
%%time
topic_cls = TopicClassifier(TMG.Layers[0])
topic_cls.train(max_num_of_topics = 3)
topics = topic_cls.classify(topic_cls.Env)

In [ ]:
%%time
TMG.create_region_layer(topics, topic_cls.tax)
logging.info(f"TMG has {len(TMG.Layers)} Layers")

### Save to files
TMG is saved as a config json file, one AnnData file per layer, and one dataframe per taxonomy. 

In [ ]:
%%time
TMG.save()